### Plot aligned variables for lrauv vehicles against combined data 
To run this Notebook follow instructions at https://github.com/mbari-org/auv-python.

Processed files must be available locally. Execute `uv run src/data/process_lrauv.py` with the `--no_cleanup` option to create them,e.g.:
```
src/data/process_lrauv.py -v --log_file pontus/missionlogs/2024/20240715_20240725/20240723T023501/202407230235_202407232319.nc4 --no_cleanup
```

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'));
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
import netCDF4 as nc4
import logging
from nc42netcdfs import BASE_LRAUV_PATH

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Change to logging.DEBUG for more detailed output
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Enable bokeh extension for hvplot
hv.extension('bokeh')

# Get time coordinate for each variable by introspection
def get_time_coord(var):
    """Get the time coordinate name for a variable.
    
    Args:
        var: Either an xarray.DataArray or netCDF4.Variable
        
    Returns:
        str: Name of the time coordinate/dimension
    """
    # Check if it's an xarray DataArray (has .dims attribute)
    if hasattr(var, 'dims'):
        # xarray DataArray
        time_dims = [dim for dim in var.dims if 'time' in dim.lower()]
        return time_dims[0] if time_dims else var.dims[0]
    elif hasattr(var, 'dimensions'):
        # netCDF4 Variable
        time_dims = [dim for dim in var.dimensions if 'time' in dim.lower()]
        return time_dims[0] if time_dims else var.dimensions[0]
    else:
        raise TypeError(f"Unsupported variable type: {type(var)}")

# Pick the auv_name
auv_name = widgets.Dropdown(
    options=[f for f in sorted(os.listdir(BASE_LRAUV_PATH)) if f != ".DS_Store"],
    description='auv_name:',
    disabled=False,
)
display(auv_name)

In [ ]:
# Pick the log file from the selected LRAUV directory
# Pattern: {lrauv_name}/missionlogs/{year}/{date_range}/{mission_start}/{log_file}.nc4

lrauv_name = auv_name.value
log_files = sorted(Path(BASE_LRAUV_PATH).glob(f"{lrauv_name}/missionlogs/*/*/*/*[0-9].nc4"))
log_file_options = [str(f.relative_to(BASE_LRAUV_PATH)) for f in log_files]

log_file_picker = widgets.Select(
    options=log_file_options,
    description='Log File:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(log_file_picker)

In [ ]:
# Load combined and aligned files for comparison
log_file = log_file_picker.value
log_path = Path(BASE_LRAUV_PATH) / log_file
log_stem = log_path.stem
log_dir = log_path.parent

# Read the combined and aligned netCDF files
combined_file = log_file.replace('.nc4', '_combined.nc4')
aligned_file = log_file.replace('.nc4', '_align.nc4')

combined_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, combined_file))
aligned_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, aligned_file))

logger.debug(f"Combined file: {combined_file}")
logger.debug(f"Aligned file: {aligned_file}")

# Show dataset details only in DEBUG mode
if logger.isEnabledFor(logging.DEBUG):
    logger.debug("\nCombined file:")
    display(combined_ds)
    logger.debug("\nAligned file:")
    display(aligned_ds)

def get_variable_coordinates(ds, variable_name):
    """Get coordinate variable names for a given data variable.
    
    Args:
        ds: xarray Dataset
        variable_name: Name of the data variable
        
    Returns:
        dict: Dictionary with keys 'longitude', 'latitude', 'depth' and coordinate names as values
    """
    coords = {}
    var = ds[variable_name]
    
    # Find coordinates by looking for common patterns in coordinate names
    for coord_name in var.coords:
        coord_lower = coord_name.lower()
        if 'lon' in coord_lower:
            coords['longitude'] = coord_name
        elif 'lat' in coord_lower:
            coords['latitude'] = coord_name
        elif 'depth' in coord_lower:
            coords['depth'] = coord_name
    
    # If coordinates not found in variable coords, check if they exist as data variables
    # (this is the case for combined files where lon/lat/depth are separate 1D variables)
    if not coords.get('longitude'):
        for var_name in ds.data_vars:
            if 'longitude' in var_name.lower():
                coords['longitude'] = var_name
                break
    if not coords.get('latitude'):
        for var_name in ds.data_vars:
            if 'latitude' in var_name.lower():
                coords['latitude'] = var_name
                break
    if not coords.get('depth'):
        for var_name in ds.data_vars:
            if 'depth' in var_name.lower():
                coords['depth'] = var_name
                break
    
    return coords

def plot_combined_vs_aligned(variable_name, plot_coords=False):
    """Plot a variable from both combined and aligned files overlaid.
    
    Args:
        variable_name: Name of the variable from combined dataset
        plot_coords: If True, also plot coordinate variables (longitude, latitude, depth)
        
    Returns:
        Overlay plot of combined and aligned data, or list of Overlay plots
    """
    logger.debug(f"Processing variable: {variable_name}")
    
    plots = []
    
    # Get time coordinates
    combined_time_coord = get_time_coord(combined_ds[variable_name])
    aligned_time_coord = get_time_coord(aligned_ds[variable_name])
    logger.debug(f"Time coords - Combined: {combined_time_coord}, Aligned: {aligned_time_coord}")
    
    # Get units for ylabel
    try:
        units = combined_ds[variable_name].attrs.get('units', '')
    except (AttributeError, KeyError):
        units = ''
    
    # Create ylabel with format "name (units)"
    ylabel = f"{variable_name} ({units})" if units else variable_name
    
    # Create time series plots with bold colors and transparency for blending
    combined_plot = combined_ds[variable_name].hvplot.line(
        x=combined_time_coord, 
        label='Combined', 
        color='#28A745',  # Green
        width=900, 
        height=400, 
        ylabel=ylabel, 
        title=log_file, 
        alpha=0.7
    )
    aligned_plot = aligned_ds[variable_name].hvplot.line(
        x=aligned_time_coord, 
        label='Aligned', 
        color='#DC3545',  # Red
        width=900, 
        height=400, 
        ylabel=ylabel, 
        title=log_file, 
        alpha=0.7
    )
    
    logger.debug("Plots created successfully")
    
    # Overlay the plots on the same axes (order determines legend order: Combined, Aligned)
    plots.append(combined_plot * aligned_plot)
    
    # Plot coordinates if requested (longitude, latitude, depth - skip time as it's the x-axis)
    if plot_coords:
        # Get coordinates for both datasets
        combined_coords = get_variable_coordinates(combined_ds, variable_name)
        aligned_coords = get_variable_coordinates(aligned_ds, variable_name)
        
        logger.debug(f"Combined coords found: {combined_coords}")
        logger.debug(f"Aligned coords found: {aligned_coords}")
        
        # Plot in order: longitude, latitude, depth
        for coord_type in ['longitude', 'latitude', 'depth']:
            combined_coord = combined_coords.get(coord_type)
            aligned_coord = aligned_coords.get(coord_type)
            
            if not combined_coord:
                logger.debug(f"No {coord_type} coordinate found in combined dataset")
                continue
            
            # Get units for coordinate
            try:
                coord_units = combined_ds[combined_coord].attrs.get('units', '')
            except (AttributeError, KeyError):
                coord_units = ''
            
            coord_ylabel = f"{coord_type} ({coord_units})" if coord_units else coord_type
            
            # Get the time coordinate for this coordinate variable
            try:
                combined_coord_time = get_time_coord(combined_ds[combined_coord])
                
                # Convert coordinate to plain DataArray to avoid naming conflicts
                combined_coord_data = xr.DataArray(
                    combined_ds[combined_coord].values,
                    coords={combined_coord_time: combined_ds[combined_coord_time]},
                    dims=[combined_coord_time],
                    name=f"{combined_coord}_data"
                )
                combined_coord_data.attrs = combined_ds[combined_coord].attrs.copy()
                
                # Create combined coordinate plot
                combined_coord_plot = combined_coord_data.hvplot.line(
                    x=combined_coord_time,
                    label='Combined',
                    color='#28A745',  # Green
                    width=900,
                    height=300,
                    ylabel=coord_ylabel,
                    title=f"{variable_name} - {coord_type}",
                    alpha=0.7
                )
                
                # Create aligned coordinate plot if available
                if aligned_coord:
                    try:
                        aligned_coord_time = get_time_coord(aligned_ds[aligned_coord])
                        
                        # Convert aligned coordinate to plain DataArray
                        aligned_coord_data = xr.DataArray(
                            aligned_ds[aligned_coord].values,
                            coords={aligned_coord_time: aligned_ds[aligned_coord_time]},
                            dims=[aligned_coord_time],
                            name=f"{aligned_coord}_data"
                        )
                        aligned_coord_data.attrs = aligned_ds[aligned_coord].attrs.copy()
                        
                        aligned_coord_plot = aligned_coord_data.hvplot.line(
                            x=aligned_coord_time,
                            label='Aligned',
                            color='#DC3545',  # Red
                            width=900,
                            height=300,
                            ylabel=coord_ylabel,
                            title=f"{variable_name} - {coord_type}",
                            alpha=0.7
                        )
                        plots.append(combined_coord_plot * aligned_coord_plot)
                    except Exception as e:
                        logger.warning(f"Could not plot aligned coordinate {aligned_coord}: {e}")
                        plots.append(combined_coord_plot)
                else:
                    plots.append(combined_coord_plot)
            except Exception as e:
                logger.warning(f"Could not plot coordinate {coord_type} ({combined_coord}): {e}")
                continue
    
    # Return list of plots instead of Layout
    return plots

# Select multiple variables to plot from the aligned dataset
variable_picker = widgets.SelectMultiple(
    options=sorted([var for var in aligned_ds.data_vars]),
    description='Variables:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(variable_picker)

# Checkbox to plot coordinates - Note: axes not linked for pan and zoom
plot_coordinates_checkbox = widgets.Checkbox(
    value=False,
    description='Plot coordinates',
    disabled=False,
    indent=False
)
display(plot_coordinates_checkbox)

In [ ]:
# Plot all selected variables with linked axes
all_plots = []
for variable_name in variable_picker.value:
    plots = plot_combined_vs_aligned(variable_name, plot_coords=plot_coordinates_checkbox.value)
    all_plots.extend(plots)

# Create layout with linked x-axes by rendering and manually linking
if all_plots:
    # Get the first plot as reference
    layout = all_plots[0]
    for plot in all_plots[1:]:
        layout = layout + plot
    
    # Use framewise=False to ensure axes stay linked across all frames (same as 4.3)
    display(layout.cols(1).opts(framewise=False, axiswise=False))
else:
    print("No plots to display")